In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_Box_sns
import GrubbsTest

In [ ]:
import copy

In [ ]:
save_dfs_to_csv = False
read_dfs_from_csv = True
save_dir_base_csvs = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_end_events_data')

assert(save_dfs_to_csv+read_dfs_from_csv <=1) # Should never both read and write!

In [ ]:
if not read_dfs_from_csv:
    conn_outages = Utilities.get_utldb01p_oracle_connection()
    conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
date_0 = '2021-01-01'
date_1 = '2021-12-31'

# date_0 = '2022-01-01'
# date_1 = '2022-05-31'

In [ ]:
save_subdir_csvs = f"{date_0.replace('-','')}_{date_1.replace('-','')}"
save_dir_csvs = os.path.join(save_dir_base_csvs, save_subdir_csvs)
if not os.path.exists(save_dir_csvs):
    os.makedirs(save_dir_csvs)

In [ ]:
cols_of_interest_met_prem = [
    'mfr_devc_ser_nbr',
    'state_cd', 
    'prem_nb',
    'trsf_pole_nb',
    'annual_kwh',
    'annual_max_dmnd', 
    'mtr_stat_cd',
    'mtr_stat_cd_desc', 
    'devc_stat_cd', 
    'devc_stat_cd_desc'
]

In [ ]:
df_outage_slim = DOVSOutages.read_df_outage_slim_from_csv(os.path.join(save_dir_csvs, 'df_outage_slim.csv'))

In [ ]:
search_time_half_window = datetime.timedelta(days=30)
#-------------------------
cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
#-----
cols_of_interest_met_prem      = TableInfos.MeterPremise_TI.std_columns_of_interest
#-------------------------
usg_split_to_CTEs=True
match_events_in_df_to_outages=False
combine_kwh_delivered_and_received=True
#-------------------------
if not usg_split_to_CTEs:
    match_events_in_df_to_outages = True
#-------------------------

In [ ]:
df_outage_slim = DOVSOutages.set_search_time_in_outage_df(
    df_outage=df_outage_slim, 
    search_time_half_window=search_time_half_window
)

In [ ]:
save_args = dict(save_to_file=False, 
                 save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_FUCK', 
                 save_name=r'end_events.csv', 
                 index=True)

# save_args = dict(save_to_file=True, 
#                  save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_prim_strict', 
#                  save_name=r'end_events.csv', 
#                  index=True)
batch_size=10
verbose=True
n_update=1

In [ ]:
df_construct_type=DFConstructType.kRunSqlQuery
contstruct_df_args_end_events=None
end_events_sql_function_kwargs = dict(
    cols_of_interest=cols_of_interest_end_dev_event, 
    df_outage=df_outage_slim, 
    split_to_CTEs=usg_split_to_CTEs, 
    join_mp_args=dict(
        join_with_CTE=True, 
        build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem), 
        join_type='LEFT'
    ), 
    df_args = dict(mapping_to_ami={'PREMISE_NBS':'premise_nbs'}, 
                   is_df_consolidated=True), 
    field_to_split='df_outage', 
    field_to_split_location_in_kwargs=['df_outage'], 
    sort_coll_to_split=True,
    batch_size=batch_size, verbose=verbose, n_update=n_update
)
addtnl_end_events_sql_function_kwargs = dict(
    build_sql_function_kwargs=dict(opco='oh')
)
end_events_sql_function_kwargs = {**end_events_sql_function_kwargs, 
                                  **addtnl_end_events_sql_function_kwargs}

# Want to re-work AMI_SQL.build_sql_ami_for_df_with_search_time_window

In [ ]:
#TODO This might make more sense if I also build a build_sql_ami_for_df method

In [ ]:
    def build_sql_ami_for_df_with_search_time_window_FINAL(
        cols_of_interest, 
        df_with_search_time_window, 
        build_sql_function=None, 
        build_sql_function_kwargs={}, 
        sql_alias_base='USG_', 
        return_args = dict(return_statement=True, 
                           insert_n_tabs_to_each_line=0), 
        final_table_alias=None, 
        prepend_with_to_stmnt=True, 
        split_to_CTEs=True, 
        max_n_for_ami_queries=None, 
        join_mp_args=False, 
        df_args={}
    ):
        r"""
        The basic idea here is that a SQL query is built using information contained inside of df_with_search_time_window.
        ------------------------- High Level Overview -------------------------
        The input parameter df_args essentially determines how df_with_search_time_window will be utilized in the construction
          and the final form of the query.
        -----
        The final query is composed of sub-queries (see below for description of general query sql_gnrl), which are dependent 
          on how df_with_search_time_window will be grouped, as dictated in df_args['addtnl_groupby_cols'].  
        The DF will always be grouped by the minimum and maximum time columns, given in df_args['t_search_min_col'] 
          and df_args['t_search_max_col'].
        Any additional columns in df_args['addtnl_groupby_cols'] will be prepended to the groupby_cols list.
        There will be (at least) one sub-query per DF group.  There can be multiple sub-queries per DF group if
          max_n_for_ami_queries is implemented in such a manner that batches are required.
        -----
        As stated above, (at least one) sub-query will be generated in each DF group.  For the explanation below, consider
          the DF corresponding to a single group in df_with_search_time_window to be sub_df.
        The information to be extracted from sub_df and inserted into the SQL query is encoded in df_args['mapping_to_ami'], 
          which is a dict whose :
              keys give columns of sub_df from which to extact data and
              values give the corresponding kwarg to use in build_sql_function to inject the data into SQL (see below for more info)
        -----
        A general query, sql_gnrl, is built first from which all sub-queries draw.  
          - If MeterPremise is to be joined (depending on join_mp_args), it will be joined at the sql_gnrl level.
        This is built using build_sql_function with cols_of_interest, join_mp_args, and build_sql_function_kwargs.
        Additionally, a WHERE statement is added containing the collection of unique date ranges for all the sub-queries.
        -----------------------------------------------------------------------
        NOTE: The datetime arguments are taken from df_with_search_time_window
                Therefore, no datetime arguments should be found in build_sql_function_kwargs.
              Similarly, df_args['mapping_to_ami'] will be taken care of for the various collections.
                Therefore, the value in df_args['mapping_to_ami'].values() should not be found in build_sql_function_kwargs
        
        Input arguments:
            cols_of_interest:
                Columns of interest to be extracted from query.
                Fed as argument to build_sql_function
            
            df_with_search_time_window:
                A datetime containing the search time window information.
            
            build_sql_function:
                Default: AMI_SQL.build_sql_ami
                NOTE: Must return a SQLQuery object (cannot simply return a string)
                
            build_sql_function_kwargs:
                Default: Listed below
                Keyword arguments for build_sql_function.
                These are used mainly for the construction of sql_gnrl, from which all sub-queries draw.
                  -Note: The sub-queries really only use this explicitly for datetime_pattern and from_table_alias,
                         although they all obviously use all build_sql_function_kwargs implicitly!
                The exact acceptable arguments depend on the function used for build_sql_function.
                Default key/value pairs:
                    serialnumber_col: 'serialnumber'
                    from_table_alias: 'un_rin'
                    date_col:         'aep_usage_dt'
                    datetime_col:     'starttimeperiod'
                    datetime_pattern: r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*"
                
            sql_alias_base:
                Default: 'USG_'
                
            return_args: 
                Default: dict(return_statement=True, insert_n_tabs_to_each_line=0)
                
            final_table_alias:
                Default: None
                
            prepend_with_to_stmnt:
                Default: True
                
            split_to_CTEs:
                Default: True
                
            max_n_for_ami_queries:
                Default: None
                The maximum number dictates how the sub-queries will further be divided into batches.
                NOTE: This can only be enforced for a single field, otherwise one would need to implement
                      a multi-dimensional batching.
                This should be a tuple/list of length two (or a single int, see below) whose 0th element
                  gives the build_sql_function input parameter to be used for batching, and whose 1st
                  element gives the maximum number per batch.
                  The 0th element must be contained in  df_args['mapping_to_ami'].values()
                  e.g., max_n_for_ami_queries = ['premise_nbs':200]
                  
                max_n_for_ami_queries as an int:
                    max_n_for_ami_queries can be an int ONLY WHEN there is a single mapping in df_args['mapping_to_ami'],
                    in which case the field to which the int value corresponds is unambiguous.
                
            join_mp_args:
                Default: False
                
            df_args:
                Default: Listed below
                Contains arguments which are used to extract the correct information from df_with_search_time_window.
                The acceptable keys for df_args are:
                    t_search_min_col:
                        Default: 't_search_min'
                        
                    t_search_max_col:
                        Default: 't_search_max'
                        
                    addtnl_groupby_cols:
                        Default: ['OUTG_REC_NB']
                        Any additional columns in df_with_search_time_window to group by.
                        The DF is always grouped by t_search_min_col and t_search_max_col.
                        When additional columns are added, these are grouped first, i.e. the time columns are always
                          grouped last.
                        These groups basically dictate how the SQL query is divided into sub-queries.  The DF 
                          df_with_search_time_window is first grouped by addtnl_groupby_cols+[t_search_min_col, t_search_max_col].
                          The groups are then iterated over, and a subquery is built using each group.
                          For each subquery, the values of addtnl_groupby_cols are included as constant values, and can be
                            identified in the ultimate resulting DF as they are tagged with '_GPD_FOR_SQL'
                        
                    is_df_consolidated:
                        Default: False
                        
                    mapping_to_ami:
                        Default: {'PREMISE_NB':'premise_nbs'}
                        This basically dictates what will be taken from df_with_search_time_window and used as inputs
                          to the queries run by build_sql_function.
                        The keys give the column in df_with_search_time_window from which to extract the information.
                        The values contain the parameter to be set in build_sql_function with the information extracted
                          from df_with_search_time_window
                        e.g., mapping_to_ami = {'PREMISE_NB':'premise_nbs'}
                            NOTE: OVERSIMPLIFIED, NOT TO BE TAKEN AS EXACT
                            NOTE: key_0                 = 'PREMISE_NB' 
                                  mapping_to_ami[key_0] = 'premise_nbs'
                            vals_from_df = df_with_search_time_window[key_0].unique()
                            --> sql = build_sql_function(... mapping_to_ami[key_0] = vals_from_df, ...)
                            
                    mapping_to_mp:
                        Default: {'STATE_ABBR_TX':'states'}
                        Only used when joining with MeterPremise (i.e., only when join_mp_args)
                        Similar in spirit to mapping_to_ami, except the extracted information is used in the MeterPremise
                          query (through join_mp_args['build_mp_kwargs'])
                          
                        This basically dictates what will be taken from df_with_search_time_window and used as inputs
                          to the queries run by AMI.join_sql_ami_with_mp.
                        The keys give the column in df_with_search_time_window from which to extract the information.
                        The values contain the parameter to be set in join_mp_args['build_mp_kwargs'] (and ultimately used
                          in AMI.join_sql_ami_with_mp) with the information extracted from df_with_search_time_window

        """
        #--------------------------------------------------
        # The whole purpose of this functionality is to take the datetime parameters from df_with_search_time_window
        # Therefore, no datetime arguments should be found in build_sql_function_kwargs
        datetime_kwargs = ['date_range', 'datetime_range']
        if len(set(datetime_kwargs).intersection(set(build_sql_function_kwargs.keys()))) != 0:
            print(f'ERROR: In build_sql_ami_for_df_with_search_time_window, datetime argument(s)={found_dts_in_build_sql_function_kwargs}'\
                  ' found in build_sql_function_kwargs\nCRASH IMMINENT')
            assert(0)
        #--------------------------------------------------
        # build_sql_function default = AMI_SQL.build_sql_ami 
        if build_sql_function is None:
            build_sql_function=AMI_SQL.build_sql_ami
        #--------------------------------------------------
        # Set up default arguments in build_sql_function_kwargs
        if build_sql_function_kwargs is None:
            build_sql_function_kwargs = {}
        build_sql_function_kwargs['serialnumber_col'] = build_sql_function_kwargs.get('serialnumber_col', 'serialnumber')
        build_sql_function_kwargs['from_table_alias'] = build_sql_function_kwargs.get('from_table_alias', 'un_rin')
        build_sql_function_kwargs['date_col']         = build_sql_function_kwargs.get('date_col', 'aep_usage_dt')
        build_sql_function_kwargs['datetime_col']     = build_sql_function_kwargs.get('datetime_col', 'starttimeperiod')
        # NOTE: Below, for datetime_pattern, using [0-9] instead of \\d (or \d) seems more robust.
        #       EMR accepts \\d but not \d, whereas Athena accepts \d but not \\d.  Both accept [0-9]
        build_sql_function_kwargs['datetime_pattern'] = build_sql_function_kwargs.get('datetime_pattern', 
                                                                                      r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*")
        #--------------------------------------------------
        # Set up default arguments in df_args.  These are used to essentially translate the information in 
        # df_with_search_time_window (i.e., used to extract the correct information from DF)
        df_args['t_search_min_col']    = df_args.get('t_search_min_col', 't_search_min')
        df_args['t_search_max_col']    = df_args.get('t_search_max_col', 't_search_max')
        df_args['addtnl_groupby_cols'] = df_args.get('addtnl_groupby_cols', ['OUTG_REC_NB'])
        df_args['is_df_consolidated']  = df_args.get('is_df_consolidated', False)
        df_args['mapping_to_ami']      = df_args.get('mapping_to_ami', {'PREMISE_NB':'premise_nbs'})
        df_args['mapping_to_mp']       = df_args.get('mapping_to_mp', {'STATE_ABBR_TX':'states'})
        #--------------------------------------------------
        # df_args['mapping_to_ami'] dictates what will be taken from df_with_search_time_window and used as inputs to the 
        #   queries run by build_sql_function.  The df_args['mapping_to_ami'] contains key,val pairs which correspond to 
        #   df_col and ami_field (see description at top).
        # Since the ami_fields will be handled for each sql_i in collection, they should be absent from build_sql_function_kwargs.
        # Also, all keys in ami_field in df_args['mapping_to_ami'] should be contained in df_with_search_time_window.columns
        for df_col, ami_field in df_args['mapping_to_ami'].items():
            assert(ami_field not in build_sql_function_kwargs)
            assert(df_col in df_with_search_time_window.columns.tolist())
        #--------------------------------------------------
        #--------------------------------------------------
        # Determine the columns by which df_with_search_time_window will be grouped.
        # The DF is always grouped by t_search_min_col and t_search_max_col.
        #   df_args['addtnl_groupby_cols'] prepends columns to the collection
        # These groups basically dictate how the SQL query is divided into sub-queries.
        #   The groups are iterated over, and a subquery is built using each group.
        #   NOTE: For each subquery, the values of addtnl_groupby_cols are included as constant values, 
        #         and can be identified in the ultimate resulting DF as they are tagged with '_GPD_FOR_SQL'
        if df_args['addtnl_groupby_cols'] is None:
            df_args['addtnl_groupby_cols'] = []
            groupby_cols = []
        elif Utilities.is_object_one_of_types(df_args['addtnl_groupby_cols'], [list, tuple]):
            # NOTE: Need to make copy of df_args['addtnl_groupby_cols'] below.
            #       Cannot simply call, e.g., groupby_cols = df_args['addtnl_groupby_cols'], as this would
            #         cause t_search_min/max to also be appended to df_args['addtnl_groupby_cols'] when they 
            #         are appended to groupby_cols, which is not desired
            #       Not sure why I was so careful here?  Doesn't really matter, as this should be harmless performance-wise
            groupby_cols = copy.deepcopy(df_args['addtnl_groupby_cols'])
        else:
            assert(isinstance(df_args['addtnl_groupby_cols'], str))
            groupby_cols = [df_args['addtnl_groupby_cols']]
        #----------
        groupby_cols.extend([df_args['t_search_min_col'], df_args['t_search_max_col']])
        #--------------------------------------------------
        # Build the grouped version of df_with_search_time_window
        # Grab both the group keys and the number of groups
        df_gpd = df_with_search_time_window.groupby(groupby_cols)
        group_keys = list(df_gpd.groups.keys())
        n_groups = df_gpd.ngroups
        assert(len(group_keys)==n_groups)
        #--------------------------------------------------
        # group_keys contains the list of groups in df_with_search_time_window.groupby(groupby_cols)
        #   The length of group_keys is the number of groups.
        #   The length of each group in group_keys should equal the length of groupby_cols, which is equal
        #     to the length of df_args['addtnl_groupby_cols'] plus 2 (for t_min and t_max)
        #   In each group, t_min and t_max are the last two elements.  Using the t_min,t_max values from all
        #     groups, build the unique date ranges.
        #   unique_date_ranges will be used in the general SQL statement, before the sub-queries.  Therefore, since
        #     there can be duplicate date ranges, and since many date ranges likely overlap, get only the unique
        #     overlap intervals for efficiency
        unique_date_ranges = [(pd.to_datetime(x[-2]).date(), pd.to_datetime(x[-1]).date()) 
                              for x in group_keys]
        unique_date_ranges = Utilities.get_overlap_intervals(unique_date_ranges)
        #--------------------------------------------------
        #--------------------------------------------------
        # Build general SQL statement, sql_gnrl
        #-------------------------

        # Need build_sql_function to return a SQLQuery object, not a string or anything else (-->return_statement=False)
        build_sql_function_kwargs['return_args'] = dict(return_statement=False)
        #-------------------------
        # If join_mp_args, use df_args['mapping_to_mp'] to fully set up join_mp_args['build_mp_kwargs']
        # NOTE: If join_mp_args is False or an empty dict, no join will occur
        # NOTE: In df_args['mapping_to_mp'], the keys give the column in df_with_search_time_window from which to 
        #       extract the information, and the values contain the parameter to be set in join_mp_args['build_mp_kwargs'] 
        #       with the information extracted
        assert(Utilities.is_object_one_of_types(join_mp_args, [bool, dict]))
        if join_mp_args:
            if isinstance(join_mp_args, bool):
                join_mp_args = {}
            if 'build_mp_kwargs' not in join_mp_args:
                join_mp_args['build_mp_kwargs'] = {}
            #-----
            for df_col, mp_field in df_args['mapping_to_mp'].items():
                join_mp_args['build_mp_kwargs'][mp_field] = df_with_search_time_window[df_col].unique().tolist()
        else:
            join_mp_args = False
        #-------------------------
        # Build sql_gnrl
        sql_gnrl = build_sql_function(cols_of_interest=cols_of_interest, 
                                      join_mp_args=join_mp_args, 
                                      **build_sql_function_kwargs)
        #-------------------------
        # If joined with MP, grab sql_mp and sql_gnrl
        if isinstance(sql_gnrl, dict):
            assert(join_mp_args) # Should only happen when joining with MP
            assert('usg_sql' in sql_gnrl)
            sql_mp   = sql_gnrl['mp_sql']
            sql_gnrl = sql_gnrl['usg_sql']
        else: 
            sql_mp = None
        #-------------------------
        # At where statements for all the date ranges in unique_date_ranges
        sql_gnrl_where = sql_gnrl.sql_where
        if sql_gnrl_where is None:
            sql_gnrl_where = SQLWhere()
        for unq_date_range in unique_date_ranges:
            sql_gnrl_where.add_where_statement(field_desc=build_sql_function_kwargs['date_col'] , comparison_operator='BETWEEN', 
                                               value=[f'{unq_date_range[0]}', f'{unq_date_range[1]}'], needs_quotes=True, 
                                               table_alias_prefix=build_sql_function_kwargs['from_table_alias'])
        sql_gnrl_where.combine_last_n_where_elements(last_n=len(unique_date_ranges), join_operator = 'OR')
        sql_gnrl.sql_where = sql_gnrl_where # Probably not necessary, but to be safe
        sql_gnrl.alias = f'{sql_alias_base}gnrl'
        #--------------------------------------------------


        #--------------------------------------------------
        # Build the collection of SQL sub-queries for each group in df_gpd
        #-------------------------
        sql_i_coll = []
        count=0 # Count is used as appendix to give each sub-query a unique alias (=f'{sql_alias_base}{count}')
        for addtnl_groupby_vals_w_t_search_min_max, sub_df in df_gpd:
            addtnl_groupby_vals = addtnl_groupby_vals_w_t_search_min_max[:-2]
            t_search_min = addtnl_groupby_vals_w_t_search_min_max[-2]
            t_search_max = addtnl_groupby_vals_w_t_search_min_max[-1]
            #----------   
            
            # Below, df_col is the column in df_with_search_time_window from which to extract the information
            #        ami_field is the parameter to be set in build_sql_function with the information extracted
            ami_input = {}
            for df_col, ami_field in df_args['mapping_to_ami'].items():
                assert(ami_field not in ami_input)
                if df_args['is_df_consolidated']:
                    assert(sub_df.shape[0]==1)
                    ami_field_values = sub_df.iloc[0][df_col]
                    assert(Utilities.is_object_one_of_types(ami_field_values, [list, tuple]))
                    ami_field_values = [x for x in ami_field_values if pd.notna(x)] # In test case, first entry had all NaN prem nbs
                else:
                    ami_field_values = sub_df[df_col].unique().tolist()
                #-----
                if len(ami_field_values)==0:
                    continue
                ami_input[ami_field] = ami_field_values
            #----------
            if not ami_input:
                continue
            #----------
            #--------------------------------------------------
            # Enforce max_n_for_ami_queries, running by batches if the collection size exceeds max_n
            batch_idxs=None
            if max_n_for_ami_queries is not None:
                assert(Utilities.is_object_one_of_types(max_n_for_ami_queries, [list, tuple, int]))
                if isinstance(max_n_for_ami_queries, int):
                    # This only makes sense if df_args['mapping_to_ami'] has only a single mapping
                    #   Otherwise, it would be uncertain as to which ami_field the max is intended for
                    assert(len(df_args['mapping_to_ami'])==1)
                    max_n_for_ami_queries = [list(df_args['mapping_to_ami'].values())[0], max_n_for_ami_queries]
                #----------
                assert(
                    Utilities.is_object_one_of_types(max_n_for_ami_queries, [list, tuple]) and 
                    len(max_n_for_ami_queries)==2 and 
                    isinstance(max_n_for_ami_queries[0], str) and 
                    isinstance(max_n_for_ami_queries[1], int)
                )
                assert(max_n_for_ami_queries[0] in df_args['mapping_to_ami'].values())
                #----------
                ami_field_to_batch = max_n_for_ami_queries[0]
                max_n              = max_n_for_ami_queries[1]
                ami_field_values_to_batch = ami_input[ami_field_to_batch]
                if len(ami_field_values_to_batch)>max_n:
                    batch_idxs = Utilities.get_batch_idx_pairs(len(ami_field_values_to_batch), max_n)
                else:
                    batch_idxs = [None]
            else:
                batch_idxs=[None]
            #--------------------------------------------------
            # Generate the sub-query.
            # If batch_idxs != [None], generate a sub-query for each batch in batch_idxs (or, I guess one could think 
            #   of it as a sub-sub-query as the query for this group is split into multiple when len(batch_idxs)>1)
            # NOTE: Made batch_idxs = [None] instead of None so I could keep all code in 'for batch_i in batch_idxs'
            #       loop regardless of whether or not batching 
            #-------------------------
            # In any case, batch_idxs should be a list
            assert(Utilities.is_object_one_of_types(batch_idxs, [list, tuple]))
            for batch_i in batch_idxs:
                assert(batch_i is None or 
                       Utilities.is_object_one_of_types(batch_i, [list, tuple]))
                if batch_i is None:
                    # Should only happen when not batching
                    assert(len(batch_idxs)==1)
                    ami_input_i = ami_input
                else:
                    assert(len(batch_i)==2)
                    i_beg = batch_i[0]
                    i_end = batch_i[1]
                    # Grab ami_input_i from ami_input, keeping only the appropriate subset of ami_field_to_batch
                    ami_input_i = {k:(v if k!=ami_field_to_batch else v[i_beg:i_end]) for k,v in ami_input.items()}
                #----------
                sql_i = build_sql_function(
                    cols_of_interest=[f"*"], 
                    date_range=[pd.to_datetime(t_search_min).date(), 
                               pd.to_datetime(t_search_max).date()], 
                    datetime_range=[t_search_min, t_search_max], 
                    datetime_pattern=build_sql_function_kwargs['datetime_pattern'], 
                    table_name=sql_gnrl.alias, 
                    schema_name=None, 
                    from_table_alias=build_sql_function_kwargs['from_table_alias'], 
                    **ami_input_i
                )
                #----------
                #sql_i.sql_from = SQLFrom(table_name=sql_gnrl.alias)
                #----------
                # For each subquery, the values of addtnl_groupby_cols are included as constant values, and can be
                #   identified in the ultimate resulting DF as they are tagged with '_GPD_FOR_SQL'
                assert(len(df_args['addtnl_groupby_cols'])==len(addtnl_groupby_vals)) # Sanity check
                for i_gpby_col in range(len(df_args['addtnl_groupby_cols'])):
                    gpby_col = df_args['addtnl_groupby_cols'][i_gpby_col]
                    gpby_col_val = addtnl_groupby_vals[i_gpby_col]
                    #TODO In build_sql_ami_for_outages, had field_desc=f"'{int(outg_rec_nb)}'"
                    sql_i.sql_select.add_select_element(field_desc=f"'{gpby_col_val}'", alias=f'{gpby_col}_GPD_FOR_SQL')
                #-----
                sql_i.alias = f'{sql_alias_base}{count}'
                #-----
                sql_i_coll.append(sql_i)
                count += 1
        #--------------------------------------------------
        return_dict = {'sql_gnrl':sql_gnrl, 
                       'sql_i_coll':sql_i_coll, 
                       'sql_mp':sql_mp}
        #--------------------------------------------------
        return_statement = return_args.get('return_statement', False)
        insert_n_tabs_to_each_line = return_args.get('insert_n_tabs_to_each_line', 0)
        prepend_with_to_stmnt = return_args.get('prepend_with_to_stmnt', True)
        if return_statement:
            return AMI_SQL.assemble_sql_ami_for_outages_statement(
                return_dict, 
                final_table_alias=final_table_alias, 
                prepend_with_to_stmnt=prepend_with_to_stmnt, 
                split_to_CTEs=split_to_CTEs
            )
        else:
            return return_dict 

In [ ]:
sql = AMIEndEvents_SQL.build_sql_end_events_for_df_with_search_time_window(
            cols_of_interest=cols_of_interest_end_dev_event, 
            df_with_search_time_window=df_outage_slim.iloc[:10], 
            build_sql_function=AMIEndEvents_SQL.build_sql_end_events, 
            build_sql_function_kwargs=dict(opco='oh'), 
            sql_alias_base='USG_', 
            return_args = dict(return_statement=True, 
                               insert_n_tabs_to_each_line=0), 
            final_table_alias=None, 
            prepend_with_to_stmnt=True, 
            split_to_CTEs=usg_split_to_CTEs, 
            max_n_prem_per_outg=None, 
            join_mp_args=dict(
                join_with_CTE=True, 
                build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem), 
                join_type='LEFT'
            ), 
            df_args = dict(mapping_to_ami={'PREMISE_NBS':'premise_nbs'}, 
                               is_df_consolidated=True)
        )

In [ ]:
print(sql)

In [ ]:
# start=time.time()
# end_events = AMIEndEvents(
#     df_construct_type=df_construct_type, 
#     contstruct_df_args = contstruct_df_args_end_events, 
#     build_sql_function=AMIEndEvents_SQL.build_sql_end_events_for_outages, 
#     build_sql_function_kwargs=end_events_sql_function_kwargs, 
#     init_df_in_constructor=True, 
#     save_args=save_args
# )
# end_events_build_time = time.time()-start

In [ ]:
cols_of_interest=cols_of_interest_end_dev_event
df_with_search_time_window=df_outage_slim.iloc[:10]
build_sql_function=AMIEndEvents_SQL.build_sql_end_events
build_sql_function_kwargs=dict(opco='oh')
sql_alias_base='USG_'
return_args = dict(return_statement=True, 
                   insert_n_tabs_to_each_line=0)
final_table_alias=None
prepend_with_to_stmnt=True
split_to_CTEs=usg_split_to_CTEs
max_n_prem_per_outg=None
join_mp_args=dict(
    join_with_CTE=True, 
    build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem), 
    join_type='LEFT'
)
df_args = dict(mapping_to_ami={'PREMISE_NBS':'premise_nbs'}, 
                   is_df_consolidated=True)

max_n_for_ami_queries=max_n_prem_per_outg

In [ ]:
build_sql_function_kwargs['serialnumber_col'] = build_sql_function_kwargs.get('serialnumber_col', 'serialnumber')
build_sql_function_kwargs['from_table_alias'] = build_sql_function_kwargs.get('from_table_alias', 'un_rin')
build_sql_function_kwargs['datetime_col']     = build_sql_function_kwargs.get('datetime_col', 'valuesinterval')
build_sql_function_kwargs['datetime_pattern'] = build_sql_function_kwargs.get('datetime_pattern', 
                                                                               r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*")
build_sql_function_kwargs['date_col']         = build_sql_function_kwargs.get('date_col', 'aep_event_dt')

In [ ]:
sql2 = AMI_SQL.build_sql_ami_for_df_with_search_time_window(
            cols_of_interest=cols_of_interest, 
            df_with_search_time_window=df_with_search_time_window, 
            build_sql_function=build_sql_function, 
            build_sql_function_kwargs=build_sql_function_kwargs, 
            sql_alias_base=sql_alias_base, 
            return_args = return_args, 
            final_table_alias=final_table_alias, 
            prepend_with_to_stmnt=prepend_with_to_stmnt, 
            split_to_CTEs=split_to_CTEs, 
            max_n_prem_per_outg=max_n_prem_per_outg, 
            join_mp_args=join_mp_args, 
            df_args = df_args
        )

In [ ]:
sql2==sql

In [ ]:
sql2b = AMI_SQL.build_sql_ami_for_df_with_search_time_window(
            cols_of_interest=cols_of_interest, 
            df_with_search_time_window=df_with_search_time_window, 
            build_sql_function=build_sql_function, 
            build_sql_function_kwargs=build_sql_function_kwargs, 
            sql_alias_base=sql_alias_base, 
            return_args = return_args, 
            final_table_alias=final_table_alias, 
            prepend_with_to_stmnt=prepend_with_to_stmnt, 
            split_to_CTEs=split_to_CTEs, 
            max_n_prem_per_outg=max_n_prem_per_outg, 
            join_mp_args=join_mp_args, 
            df_args = df_args
        )

In [ ]:
sql2b==sql2

In [ ]:
sql3 = build_sql_ami_for_df_with_search_time_window_FINAL(
            cols_of_interest=cols_of_interest, 
            df_with_search_time_window=df_with_search_time_window, 
            build_sql_function=build_sql_function, 
            build_sql_function_kwargs=build_sql_function_kwargs, 
            sql_alias_base=sql_alias_base, 
            return_args = return_args, 
            final_table_alias=final_table_alias, 
            prepend_with_to_stmnt=prepend_with_to_stmnt, 
            split_to_CTEs=split_to_CTEs, 
            max_n_for_ami_queries=max_n_for_ami_queries, 
            join_mp_args=join_mp_args, 
            df_args = df_args
        )

In [ ]:
sql3==sql2

In [ ]:
df_outage_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_outage_OG.csv'), dtype=str)
csv_cols_and_types_to_convert_dict = {'CI_NB':np.int32, 'CMI_NB':np.float64, 'OUTG_REC_NB':[np.float64, np.int32]}
df_outage_OG = Utilities_df.convert_col_types(df_outage_OG, csv_cols_and_types_to_convert_dict)

In [ ]:
df_outage = df_outage_OG.copy()

In [ ]:
search_time_half_window = datetime.timedelta(days=30)
df_outage = DOVSOutages.set_search_time_in_outage_df(
    df_outage=df_outage, 
    search_time_half_window=search_time_half_window
)

In [ ]:
df_outage

In [ ]:
cols_of_interest=cols_of_interest_end_dev_event
df_with_search_time_window=df_outage[df_outage['OUTG_REC_NB'].isin(df_outage_slim.iloc[:10]['OUTG_REC_NB'].unique().tolist())]
build_sql_function=AMIEndEvents_SQL.build_sql_end_events
build_sql_function_kwargs=dict(opco='oh')
sql_alias_base='USG_'
return_args = dict(return_statement=True, 
                   insert_n_tabs_to_each_line=0)
final_table_alias=None
prepend_with_to_stmnt=True
split_to_CTEs=usg_split_to_CTEs
max_n_prem_per_outg=None
join_mp_args=dict(
    join_with_CTE=True, 
    build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem), 
    join_type='LEFT'
)
df_args = dict(mapping_to_ami={'PREMISE_NB':'premise_nbs'}, 
                   is_df_consolidated=False)

max_n_for_ami_queries=max_n_prem_per_outg

In [ ]:
build_sql_function_kwargs['serialnumber_col'] = build_sql_function_kwargs.get('serialnumber_col', 'serialnumber')
build_sql_function_kwargs['from_table_alias'] = build_sql_function_kwargs.get('from_table_alias', 'un_rin')
build_sql_function_kwargs['datetime_col']     = build_sql_function_kwargs.get('datetime_col', 'valuesinterval')
build_sql_function_kwargs['datetime_pattern'] = build_sql_function_kwargs.get('datetime_pattern', 
                                                                               r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*")
build_sql_function_kwargs['date_col']         = build_sql_function_kwargs.get('date_col', 'aep_event_dt')

In [ ]:
sql4 = build_sql_ami_for_df_with_search_time_window_FINAL(
            cols_of_interest=cols_of_interest, 
            df_with_search_time_window=df_with_search_time_window, 
            build_sql_function=build_sql_function, 
            build_sql_function_kwargs=build_sql_function_kwargs, 
            sql_alias_base=sql_alias_base, 
            return_args = return_args, 
            final_table_alias=final_table_alias, 
            prepend_with_to_stmnt=prepend_with_to_stmnt, 
            split_to_CTEs=split_to_CTEs, 
            max_n_for_ami_queries=max_n_for_ami_queries, 
            join_mp_args=join_mp_args, 
            df_args = df_args
        )

In [ ]:
print(sql4)

In [ ]:
sql4==sql

In [ ]:
SNs = DOVSOutages.get_serial_numbers_for_outages(
    outg_rec_nbs=df_outage_slim['OUTG_REC_NB'].unique().tolist(), 
    return_premise_nbs_for_outages=True
)

In [ ]:
SNs.keys()

In [ ]:
SNs_df = pd.DataFrame(SNs)
SNs_df.index = SNs_df.index.astype(int)

In [ ]:
SNs_df

In [ ]:
df_outage_slim

In [ ]:
pd.merge(df_outage_slim, SNs_df, left_index=True, right_index=True, how='inner')